I still remember my first day in machine learning class. The first example which was provided to explain, how machine learning works, was “Spam Detection”. I think in most of the machine learning courses tutors provide the same example, but, in how many courses you actually get to implement the model? We talk how machine learning involved in Spam Detection and then just move on to other things.

Introduction

The idea of this post is to understand step by step working of the spam filter and how it helps in making everyone life easier. Also, next time when you see a “You have won a lottery” email rather than ignoring it, you might prefer to report it as a spam.

# Gmail Spam Detection

We all know the data Google has, is not obviously in paper files. They have data centers which maintain the customers data. Before Google/Gmail decides to segregate the emails into spam or not spam category, before it arrives to your mailbox, hundreds of rules apply to those email in the data centers. These rules describe the properties of a spam email. 
There are common types of spam filters which are used by Gmail/Google —

# Blatant Blocking- 
Deletes the emails even before it reaches to the inbox.
# Bulk Email Filter- 
This filter helps in filtering the emails that are passed through other categories but are spam.
# Category Filters- 
User can define their own rules which will enable the filtering of the messages according to the specific content or the email addresses etc.
# Null Sender Disposition- 
Dispose of all messages without an SMTP envelope sender address. Remember when you get an email saying, “Not delivered to xyz address”.
# Null Sender Header Tag Validation-
Validate the messages by checking security digital signature.

There are ways to avoid spam filtering and send your emails straight to the inbox. To learn more about Gmail spam filter please watch this informational video from Google.

# Create a Spam Detector : Pre-processing

Moving on to our aim of creating our very own spam detector. Let’s talk about about that blue box in the middle of above image. The model is like a small kid unless you tell the kid, the difference between salt and sugar, he/she won’t be able to recognize it. The similar idea we apply on machine learning model, we tell the model beforehand what kind of email can be spam or not spam. In order to do that we need to collect the data from users and ask them to filter few emails as spam or not spam.

In [1]:
import numpy as np
import pandas as pd
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
#Run the below piece of code for the first time
#nltk.download('stopwords')

In [2]:
message_data = pd.read_csv(r"C:\Users\ankit\Desktop\DATA.S\Spam\spam.csv",encoding = "latin")
message_data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [3]:
message_data = message_data.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis = 1)

In [4]:
message_data = message_data.rename(columns = {'v1':'Spam/Not_Spam','v2':'message'})

In [5]:
message_data.head()

,Spam/Not_Spam,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
message_data.groupby('Spam/Not_Spam').describe()

message         \
                count unique   
Spam/Not_Spam                  
ham              4825   4516   
spam              747    653   

                                                                       
                                                             top freq  
Spam/Not_Spam                                                          
ham                                       Sorry, I'll call later   30  
spam           Please call our customer service representativ...    4

In [7]:
message_data_copy = message_data['message'].copy()

In [8]:
message_data_copy

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
5       FreeMsg Hey there darling it's been 3 week's n...
6       Even my brother is not like to speak with me. ...
7       As per your request 'Melle Melle (Oru Minnamin...
8       WINNER!! As a valued network customer you have...
9       Had your mobile 11 months or more? U R entitle...
10      I'm gonna be home soon and i don't want to tal...
11      SIX chances to win CASH! From 100 to 20,000 po...
12      URGENT! You have won a 1 week FREE membership ...
13      I've been searching for the right words to tha...
14                    I HAVE A DATE ON SUNDAY WITH WILL!!
15      XXXMobileMovieClub: To use your credit, click ...
16                             Oh k...i'm watching here:)
17      Eh u r

We will first do some pre-processing on message text, like removing - punctuation and stop words.

In [9]:
def text_preprocess(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = [word for word in text.split() if word.lower() not in stopwords.words('english')]
    return " ".join(text)

In [10]:
message_data_copy = message_data_copy.apply(text_preprocess)

In [11]:
message_data_copy

0       Go jurong point crazy Available bugis n great ...
1                                 Ok lar Joking wif u oni
2       Free entry 2 wkly comp win FA Cup final tkts 2...
3                     U dun say early hor U c already say
4             Nah dont think goes usf lives around though
5       FreeMsg Hey darling 3 weeks word back Id like ...
6          Even brother like speak treat like aids patent
7       per request Melle Melle Oru Minnaminunginte Nu...
8       WINNER valued network customer selected receiv...
9       mobile 11 months U R entitled Update latest co...
10      Im gonna home soon dont want talk stuff anymor...
11      SIX chances win CASH 100 20000 pounds txt CSH1...
12      URGENT 1 week FREE membership å£100000 Prize J...
13      Ive searching right words thank breather promi...
14                                            DATE SUNDAY
15      XXXMobileMovieClub use credit click WAP link n...
16                                        Oh kim watching
17      Eh u r

Once the pre-processing is done, we would need to vectorize the data — i.e collecting each word and its frequency in each email. The vectorization will produce a matrix.

In [13]:
vectorizer = TfidfVectorizer("english")

In [14]:
message_mat = vectorizer.fit_transform(message_data_copy)
message_mat

<5572x9376 sparse matrix of type '<class 'numpy.float64'>'
	with 47254 stored elements in Compressed Sparse Row format>

This vector matrix can be used create train/test split. This will help us to train the model/machine to be smart and test the accuracy of its results.

In [15]:
message_train, message_test, spam_nospam_train, spam_nospam_test = train_test_split(message_mat, 
                                                        message_data['Spam/Not_Spam'], test_size=0.3, random_state=20)

# Choosing a model
Now that we have train test split, we would need to choose a model. There is a huge collection of models but for this particular exercise we will be using logistic regression.Why?

Generally when someone asks, what is logistic regression? what do you tell them — Oh! it is an algorithm which is used for categorizing things into two classes (most of the time) i.e. the result is measured using a dichotomous variable. But, how does logistic regression classify thing into classes like -binomial(2 possible values), multinomial(3 or more possible values) and ordinal(deals with ordered categories). For this post we will only be focusing on binomial logistic regression i.e. the outcome of the model will be categorized into two classes.

Logistic Regression

According to Wikipedia definition,
Logistic Regression measures the relationship between the categorical dependent variable and one or more independent variables by estimating probabilities using a logistic function.

From the definition it seems, the logistic function plays an important role in classification here but we need to understand what is logistic function and how does it help in estimating the probability of being in a class.

                         phi(z)=1/1+exponential(-z)

The formula mentioned in the above image is known as Logistic function or Sigmoid function and the curve called Sigmoid curve. The Sigmoid function gives an S shaped curve. The output of Sigmoid function tends towards 1 as z → ∞ and tends towards 0 as z → −∞. Hence Sigmoid/logistic function produces the value of dependent variable which will always lie between [0,1] i.e the probability of being in a class.

# Modelling
For the Spam detection problem, we have tagged messages but we are not certain about new incoming messages. We will need a model which can tell us the probability of a message being Spam or Not Spam. Assuming in this example , 0 indicates — negative class (absence of spam) and 1 indicates — positive class (presence of spam), we will use logistic regression model.

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

Spam_model = LogisticRegression(solver='liblinear', penalty='l1')
Spam_model.fit(message_train, spam_nospam_train)
pred = Spam_model.predict(message_test)
accuracy_score(spam_nospam_test,pred)

0.9383971291866029

So, first we define the model then fit the train data — this phase is called training your model. Once the training phase is finished we can use the test split and predict the results. In order to check the accuracy of our model we can use accuracy score metric. This metric compares the predicted results with the obtained true results. After running above code we got 93% accuracy.
In some cases 93% might seems a good score. There are a lot other things we can do with the collected data in order to achieve more accurate results, like stemming the words and normalizing the length.

# Let's try using stemming and normalizing length of the messages

In [17]:
def stemmer (text):
    text = text.split()
    words = ""
    for i in text:
            stemmer = SnowballStemmer("english")
            words += (stemmer.stem(i))+" "
    return words

In [18]:
message_data_copy = message_data_copy.apply(stemmer)
vectorizer = TfidfVectorizer("english")
message_mat = vectorizer.fit_transform(message_data_copy)

In [19]:
message_train, message_test, spam_nospam_train, spam_nospam_test = train_test_split(message_mat, 
                                                        message_data['Spam/Not_Spam'], test_size=0.3, random_state=20)

In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

Spam_model = LogisticRegression(solver='liblinear', penalty='l1')
Spam_model.fit(message_train, spam_nospam_train)
pred = Spam_model.predict(message_test)
accuracy_score(spam_nospam_test,pred)

0.9461722488038278

# Accuracy score improved. Let's try normalizing length.

In [21]:
message_data['length'] = message_data['message'].apply(len)
message_data.head()

,Spam/Not_Spam,message,length
0,ham,"Go until jurong point, crazy.. Available only ...",111
1,ham,Ok lar... Joking wif u oni...,29
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155
3,ham,U dun say so early hor... U c already then say...,49
4,ham,"Nah I don't think he goes to usf, he lives aro...",61


In [22]:
length = message_data['length'].as_matrix()
new_mat = np.hstack((message_mat.todense(),length[:, None]))

C:\Users\ankit\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [23]:
message_train, message_test, spam_nospam_train, spam_nospam_test = train_test_split(new_mat, 
                                                        message_data['Spam/Not_Spam'], test_size=0.3, random_state=20)

In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

Spam_model = LogisticRegression(solver='liblinear', penalty='l1')
Spam_model.fit(message_train, spam_nospam_train)
pred = Spam_model.predict(message_test)
accuracy_score(spam_nospam_test,pred)

0.9467703349282297

# Summary
As we saw, we used previously collected data in order to train the model and predicted the category for new incoming emails. This indicate the importance of tagging the data in right way. One mistake can make your machine dumb, e.g In your gmail or any other email account when you get the emails and you think it is a spam but you choose to ignore, may be next time when you see that email, you should report that as a spam. This process can help a lot of other people who are receiving the same kind of email but not aware of what spam is. Sometimes wrong spam tag can move a genuine email to spam folder too. So, you have to be careful before you tag an email as a spam or not spam.